In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install faker

In [1]:
import numpy as np
import pandas as pd
import json
import pickle
from tqdm import tqdm
import warnings
import os
#from random import *
import random
from faker import Faker
import re

warnings.filterwarnings('ignore')

In [2]:
def json_to_pickle(data_path, json_file, pickle_file):
    
    # load json file
    with open(data_path + json_file + '.json', 'r', encoding='utf-8') as f:
        json_data = json.load(f)
        
    with open(data_path + pickle_file +'.pkl', 'wb') as f:
        pickle.dump(json_data, f, protocol=pickle.HIGHEST_PROTOCOL)

In [3]:
def concat_files(file_ex:str, data_path):
    def get_text(data_path, file_name):
        with open(data_path+file_name, 'rb') as f:
            data = pickle.load(f)

        # json -> dict
        temp_dict = {}
        for i in tqdm(range(len(data['data']))):
            temp_txt_list = []
            for j in range(len(data['data'][i]['body'])):
                temp_txt = data['data'][i]['body'][j]['utterance']
                temp_txt_list.append(temp_txt)

            temp_dict[i] = temp_txt_list


        # join messages
        temp_list = []
        for i in range(len(temp_dict)):
            temp_txt = ' '.join(temp_dict[i])
            temp_list.append(temp_txt)

        df = pd.DataFrame(temp_list, columns=['text'])

        return df
    
    pkl_list = [file for file in os.listdir(data_path) if file.endswith(file_ex)]
    pkl_list

    name_list = []
    for l in pkl_list:
        name_list.append(l.split('.')[0])

    for a in name_list:
        globals()[f'{a}'] = get_text(data_path, f'{a}'+file_ex)

    df_concat = pd.concat([globals()[f'{name_list[i]}'] for i in range(len(name_list))], ignore_index=True)

    return df_concat

In [4]:
data_path = './KB_data/korean_sns/Training/'
#data_path = './KB_data/korean_sns/Validation/'

In [5]:
json_list = ['미용과건강','상거래(쇼핑)','시사교육','식음료','여가생활','일과직업','주거와생활','행사','개인및관계']
pickle_list = ['beauty_health','shopping','education','food','leisure','work','living','events','personal_relationship']

for i in range(len(json_list)):
    json_to_pickle(data_path, json_list[i], pickle_list[i])

In [6]:
concat_file = concat_files('.pkl', data_path)

with open(data_path + 'concat_file.pkl', 'wb') as f:
        pickle.dump(concat_file, f, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
with open(data_path + 'concat_file.pkl', 'rb') as f:
        concat_data = pickle.load(f)

In [6]:
tag_data = concat_data[concat_data['text'].str.contains('#@')]
tag_data.reset_index(drop=True, inplace=True)

In [7]:
filtered_text=[]

text=tag_data.text.values

tag_list=["#@이름#"
          ,"#@계정#"
          ,"#@신원#"
          ,"#@전번#"
          ,"#@금융#"
          ,"#@번호#"
          ,"#@주소#"]

for t in text:
    for tag in tag_list:
        if tag in t:
            filtered_text.append(t)

In [8]:
filtered_df=pd.DataFrame(filtered_text,columns=["text"])
filtered_df.head()

,text
0,구냥 기른거같은데... ㅋ켘캨ㅋㅋㅋ ㅋㅋㅋㅋㅋㅋ알지 뭔날린디 #@이름# 털을보면 관...
1,앜ㅋㅋㅋㅋㅋㅋ나 머리가 저렇게 짧았나ㅋㅋㅋㅋㅋㅋ 레이어드컷 언니가 혹할만한 예쁜 사...
2,옥수수 털고싶다 옥수수 뽑아줄까 #@이름#센세는 옥수수 한번 뽑았었음 님도 곧 사랑...
3,#@이름# 아니겠죠...독감아닌데 열이 어젯밤도 안 떨어진다더라구요 그래서 오늘 #...
4,귓불도 뚫을 수 있으면 좋은데 아님 좀 윗쪽으로해서 함 뚫어 앞에서 봤을때 보이게....


In [9]:
def count_tags(text) :
    name = []
    address = []
    tag_dict={'#@이름#': 0  #성+이름(1), 이름(9)
              ,'#@계정#': 0  #이메일
              ,'#@신원#': 0 #주민번호
              ,'#@전번#': 0 #핸드폰번호
              ,'#@금융#': 0  #계좌번호(국민) (은행명 계좌번호) , (계좌번호 은행명), (oo은행 , oo)
              ,'#@번호#': 0 # 4~6자리 , 5:5
              ,'#@주소#': 0} # 동이하 주소, 지번

    for t in text:
        for tag in tag_list:
            if tag in t:
                tag_dict[tag]+=1

    return tag_dict

In [ ]:
filtered_text=set(filtered_text)
print(f'filtered_all : {len(filtered_text)}')
tag_dict = count_tags(filtered_text)
print(tag_dict)

In [11]:
no_name = set()
tag_list=[
"#@계정#",
"#@신원#",
"#@전번#",
"#@금융#",
"#@번호#",
"#@주소#",
]

for text in filtered_text:
    for tag in tag_list:
        if tag in text:
            no_name.add(text)

In [12]:
tag_list=["#@이름#"
          ,"#@계정#"
          ,"#@신원#"
          ,"#@전번#"
          ,"#@금융#"
          ,"#@번호#"
          ,"#@주소#"]

count_tags(no_name)

{'#@이름#': 5790,
 '#@계정#': 2835,
 '#@신원#': 483,
 '#@전번#': 1604,
 '#@금융#': 1819,
 '#@번호#': 2942,
 '#@주소#': 4480}

### preprocessing

In [13]:
#  '#@이름#': 0, #성+이름(1), 이름(9)
# 이름 list 생성
fake = Faker('ko-KR')

def has_coda(word):
    return (ord(word[-1])-44032)%28==0
             
#  '#@계정#': 0,  #이메일
def create_email(iter:int):
    
    email_list = []
    
    for _ in range(iter):
    
        email = ""
        
        string_pool = "abcdedfhijklmnopqrstuvwxyz"
        number_pool = "1234567890"
        domain_pool = [".com", ".kr", ".net", "co.kr", ".ac.kr"]
        
        for l in range(random.randint(5,20)) :
            email += random.choice(string_pool+number_pool)

        email += "@"

        for l in range(random.randint(5,10)) :
            email += random.choice(string_pool)
            
        email += random.choice(domain_pool)
        
        email_list.append(email)

    return email_list
    
#  '#@신원#': 0, #주민번호
def create_id_code(iter:int, proportion=0.5):
    id_code_list = []

    for i in range(int(iter*proportion)):
        number = str(int(random.random()*1000000)).ljust(6, '0')+'-'+str(int(random.random()*10000000)).ljust(7, '0')
        id_code_list.append(number)

    for i in range(int(iter*(1-proportion))):
        number = str(int(random.random()*1000000)).ljust(6, '0')+' '+str(int(random.random()*10000000)).ljust(7, '0')
        id_code_list.append(number)

    return id_code_list

#  '#@전번#': 0, #핸드폰번호
def create_phone_number(iter:int, proportion=0.5):
    phone_number_list = []
    
    for i in range(int(iter*proportion)):
        number = '010'+'-'+str(int(random.random()*10000)).zfill(4)+'-'+str(int(random.random()*10000)).zfill(4)
        phone_number_list.append(number)

    for i in range(int(iter*(1-proportion))):
        number = '010'+str(int(random.random()*10000)).zfill(4)+str(int(random.random()*10000)).zfill(4)
        phone_number_list.append(number)

    return phone_number_list

#  '#@금융#': 0, #계좌번호(국민) (은행명 계좌번호) , (계좌번호 은행명), (oo은행 , oo)
def create_account(iter:int):
      
    ''''
    dash : 계좌에 "-" 추가
    bank_first : 은행명이 제일 앞에, False시 제일 뒤에
    short: True시 "꾹민" False 시 "국민은행"
    '''
    
    account_list = []
    
    for _ in range(iter):
        account = ""
        dash=random.choice([True,False])
        bank_first=random.choice([True,False])
        short=random.choice([True,False])
        
        if dash:
            while len(account)<14:
                account+=str(random.randint(0,9))
                
            account=account[:6]+"-"+account[6:8]+"-"+account[8:]
            
        else:
            while len(account)<14:
                account+=str(random.randint(0,9))
                
        
        if short:
            if bank_first:
                account = "국민 " + account
            else:
                account = account + " 국민"
                
        else:
            if bank_first:
                account ="국민은행 "  +account
                
            else:
                account = account + " 국민은행"
        
        account_list.append(account)      
     
    return account_list      

#  '#@번호#': 0, # 4~6자리 , 5:5
def create_pw(iter:int, proportion=0.5):
    pw_list = []
    
    for i in range(int(iter*proportion)):
        pw = str(int(random.random()*10000)).zfill(4)
        pw_list.append(pw)
    
    for i in range(int(iter*(1-proportion))):
        pw = str(int(random.random()*1000000)).zfill(6)
        pw_list.append(pw)

    return pw_list

#  '#@주소#': 0} # 동이하 주소, 지번
def create_address(iter:int):
    from faker import Faker
    fake = Faker('ko-KR')

    address_list = []
    for i in range(iter):
        faker_address = fake.address()
        if '(' in faker_address:
            pass
        else:
            address_list.append(faker_address+' '+str(int(random.random()*1000)))

    return address_list


In [14]:
def convert_tag_to_name(sentence:str,target="#@이름#",name1_list=None,name2_list=None):
    
    ## 이름 자동생성기 찾아야 함
    
    
    flag=True
    position1=["아","이","은"] #받침 o
    position2=["야","가","는"] #받침 x

    full_name_list=name1_list+name2_list
    
    
    
    while flag:
        
        start_index=sentence.find(target)
        end_index=start_index+5
        
        if start_index==-1:
            flag=False
            
        else:

            try:
                if sentence[end_index] in position1:

                    # print("aaaaa : ", sentence[end_index])
                    
                    # name=random.choice(name1_list)
                    # idx=random.randint(0,len(name1_list))
                    name=name1_list.pop()
                    # name=name1_list[idx]

                    last_name=random.choice([True,False])
                    
                    if not last_name:
                        name=name[1:]
                    
                    # print(sentence[end_index])
                    if start_index == 0:
                        sentence=name + sentence[end_index:]
                    else:
                        sentence=sentence[:start_index] + name + sentence[end_index:]
                        
                        
                elif sentence[end_index] in position2:
                    
                    # name=random.choice(name2_list)
                    name=name2_list.pop()
                    last_name=random.choice([True,False])
                    
                    if not last_name:
                        name=name[1:]
                    
                    # print(sentence[end_index])
                    if start_index == 0:
                        sentence=name + sentence[end_index:]
                    else:
                        sentence=sentence[:start_index] + name + sentence[end_index:]
                        
                else:
                    
                    # name=random.choice(name1_list+name2_list)
                    name = full_name_list.pop()
                    last_name=random.choice([True,False])
                    
                    if not last_name:
                        name=name[1:]
                    
                    # print(sentence[end_index])
                    sentence= sentence[:start_index] + name + sentence[end_index:]

            except IndexError:
                    # name=random.choice(name1_list+name2_list)
                    name=full_name_list.pop()
                    last_name=random.choice([True,False])
                    
                    if not last_name:
                        name=name[1:]
                    
                    # print(sentence[end_index])
                    sentence=sentence[:start_index] + name + sentence[end_index:]
                    
    return sentence

def convert_tag(sentence:str,target=None,list_name=None):
    
    flag=True
    
    
    while flag:
        
        start_index=sentence.find(target)
        end_index=start_index+5
        
        if start_index==-1:
            flag=False
            
        else:
            data=random.choice(list_name)

            sentence=sentence[:start_index] + data + sentence[end_index:]
     
                
    return sentence  

In [ ]:
#create random data list

name_list_coda=[] #받침 o
name_list_nocoda=[] #받침 x

for _ in range(150000):
    
    name = fake.name()
    
    if has_coda(name):
        name_list_nocoda.append(name)
    else:
        name_list_coda.append(name)
    
iter = 5000
email_list = create_email(iter)
id_code_list = create_id_code(iter)
phone_number_list = create_phone_number(iter)
account_list = create_account(iter)
pw_list = create_pw(iter)
address_list = create_address(iter)


with open("name_list_coda.list","wb") as f:
    pickle.dump(name_list_coda,f)  
with open("name_list_nocoda.list","wb") as f:
    pickle.dump(name_list_nocoda,f)
with open("email_list.list","wb") as f:
   pickle.dump(email_list,f)
with open("id_code_list.list","wb") as f:
   pickle.dump(id_code_list,f)
with open("phone_number_list.list","wb") as f:
   pickle.dump(phone_number_list,f)
with open("account_list.list","wb") as f:
   pickle.dump(account_list,f)
with open("pw_list.list","wb") as f:
   pickle.dump(pw_list,f)
with open("address_list.list","wb") as f:
   pickle.dump(address_list,f)

In [17]:
no_name_df=pd.DataFrame(no_name,columns=["text"])
no_name_df.head()

,text
0,나 폰 바꿨음 번호도 바꼈삼 그런 거 같네 카톡 이름 바뀜 번호 갈켜줘 #@전번# ...
1,인스타 보면 얼굴 나온다 머라쳐?ㅋ구경좀해야겠다 어떤가 얼굴이랑 매치되나 직접칠래?...
2,집으로 갈꺼야? 울집으로 와랑 ㅋㅋㅋㅋ자기네집가라고? 택시타고 오케와 택시가 이써?...
3,알겠다리zzz 언니 두타 아듸 뭐야 두타? 엉 #@계정# 비번은 #@번호# 가방두타...
4,#@URL# 여기로 접근하셔서 이름: #@계정# 비번: #@번호# 이용하시면 홈페이...


In [18]:
split_text= no_name_df['text'].apply(lambda x:x.split()).values

In [53]:
if __name__== "__main__":
    data_path = './'
    file_ex = '.list'
    data_list = [file for file in os.listdir(data_path) if file.endswith(file_ex)]
    for data in data_list :
        with open(data,'rb') as f :
            globals()[f'{data[:-5]}'] = pickle.load(f)  
    
    tag_dict={"#@이름#": ""
            ,"#@계정#": email_list
            ,"#@신원#": id_code_list
            ,"#@전번#" : phone_number_list
            ,"#@금융#" : account_list
            ,"#@번호#" : pw_list
            ,"#@주소#" : address_list} 
    
    result = []
    labels=[]

    target="#@[ㄱ-힣a-zA-Z0-9]+#"
    
    for li in tqdm(split_text):
        sentence = []
        label=[]
        for token in li:
            new_token =re.search(target, token)
            try : 
                    
                if token.find("#@") == -1  :
                    sentence.append(token)
                    label.append('o')
                
                elif new_token.group() not in tag_dict :
                    sentence.append(token)
                    label.append('o')

                elif new_token.group() in tag_dict:
                    for tag in tag_dict :
                        if tag == "#@이름#" :
                            converted_token = convert_tag_to_name(token, tag, name_list_coda, name_list_nocoda)
                        else :
                            converted_token = convert_tag(token, tag, tag_dict[tag])
                        
                        if token != converted_token :
                            sentence.append(converted_token)
                            label.append(tag)
            except :
                pass
                
        result.append(sentence)
        labels.append(label)

            
    print(len(result), len(labels))

with open("./result.list","wb") as f:
   pickle.dump(result,f)
with open("./labels.list","wb") as f:
   pickle.dump(labels,f)

100%|██████████| 12644/12644 [16:30<00:00, 12.76it/s]


12644 12644


In [ ]:
with open('./KB_data/result.list', 'rb') as f:
#with open('./KB_data/result_val.list', 'rb') as f:
    data = pickle.load(f)
with open('./KB_data/labels.list', 'rb') as f:
#with open('./KB_data/labels_val.list', 'rb') as f:
  labels = pickle.load(f)

joined_data = []

for a in data:
    temp = ' '.join(a)
    joined_data.append(temp)

joined_label = []

for a in labels:
    temp = ' '.join(a)
    joined_label.append(temp)

df = pd.DataFrame(data=list(zip(joined_data, joined_label)))

df.to_csv('./KB_data/train.tsv', index=False, header=None, sep='\t')
#df.to_csv('./KB_data/test.tsv', index=False, header=None, sep='\t')